In [15]:
import pyceres
import pycolmap
import numpy as np
from hloc.utils import viz_3d

from pycolmap import cost_functions

## Synthetic reconstruction

In [16]:
def create_reconstruction(num_points=50, num_images=2, seed=3, noise=0):
    state = np.random.RandomState(seed)
    rec = pycolmap.Reconstruction()
    p3d = state.uniform(-1, 1, (num_points, 3)) + np.array([0, 0, 3])
    for p in p3d:
        rec.add_point3D(p, pycolmap.Track(), np.zeros(3))
    w, h = 640, 480
    cam = pycolmap.Camera(
        model="SIMPLE_PINHOLE",
        width=w,
        height=h,
        params=np.array([max(w, h) * 1.2, w / 2, h / 2]),
        camera_id=0,
    )
    rec.add_camera(cam)
    for i in range(num_images):
        im = pycolmap.Image(
            id=i,
            name=str(i),
            camera_id=cam.camera_id,
            cam_from_world=pycolmap.Rigid3d(
                pycolmap.Rotation3d(), state.uniform(-1, 1, 3)
            ),
        )
        im.registered = True
        p2d = cam.img_from_cam(
            im.cam_from_world * [p.xyz for p in rec.points3D.values()]
        )
        p2d_obs = np.array(p2d) + state.randn(len(p2d), 2) * noise
        im.points2D = pycolmap.ListPoint2D(
            [pycolmap.Point2D(p, id_) for p, id_ in zip(p2d_obs, rec.points3D)]
        )
        rec.add_image(im)
    return rec


rec_gt = create_reconstruction()

In [17]:
fig = viz_3d.init_figure()
viz_3d.plot_reconstruction(
    fig, rec_gt, min_track_length=0, color="rgb(255,0,0)", points_rgb=False
)
fig.show()

## Optimize 3D points

In [18]:
def define_problem(rec): #ORIGINAL
    prob = pyceres.Problem()
    loss = pyceres.TrivialLoss()
    for im in rec.images.values():
        cam = rec.cameras[im.camera_id]
        for p in im.points2D:
            cost = pycolmap.cost_functions.ReprojErrorCost(
                cam.model, im.cam_from_world, p.xy
            )
            prob.add_residual_block(
                cost, loss, [rec.points3D[p.point3D_id].xyz, cam.params]
            )
    for cam in rec.cameras.values():
        prob.set_parameter_block_constant(cam.params)
    return prob

def solve(prob):
    print(
        prob.num_parameter_blocks(),
        prob.num_parameters(),
        prob.num_residual_blocks(),
        prob.num_residuals(),
    )
    options = pyceres.SolverOptions()
    options.linear_solver_type = pyceres.LinearSolverType.DENSE_QR
    options.minimizer_progress_to_stdout = True
    options.num_threads = -1
    summary = pyceres.SolverSummary()
    pyceres.solve(options, prob, summary)
    print(summary.BriefReport())

In [19]:
rec = create_reconstruction()
problem = define_problem(rec)
solve(problem)

51 153 100 200
iter      cost      cost_change  |gradient|   |step|    tr_ratio  tr_radius  ls_iter  iter_time  total_time
   0  5.119392e-26    0.00e+00    3.16e-11   0.00e+00   0.00e+00  1.00e+04        0    2.97e-04    1.30e-03
Ceres Solver Report: Iterations: 1, Initial cost: 5.119392e-26, Final cost: 5.119392e-26, Termination: CONVERGENCE


Add some noise

In [20]:
rec = create_reconstruction()
for p in rec.points3D.values():
    p.xyz += np.random.RandomState(0).uniform(-0.5, 0.5, 3)
print(rec.points3D[1].xyz)
problem = define_problem(rec)
solve(problem)

[0.15040931 0.63148501 2.68457285]
51 153 100 200
iter      cost      cost_change  |gradient|   |step|    tr_ratio  tr_radius  ls_iter  iter_time  total_time
   0  1.074601e+05    0.00e+00    3.34e+04   0.00e+00   0.00e+00  1.00e+04        0    2.31e-04    9.33e-04
   1  1.048585e+02    1.07e+05    1.33e+03   1.77e+00   9.99e-01  3.00e+04        1    2.33e-03    3.29e-03
   2  8.359400e-05    1.05e+02    1.50e+00   4.77e-02   1.00e+00  9.00e+04        1    2.20e-03    5.51e-03
   3  2.083842e-14    8.36e-05    1.75e-05   4.03e-05   1.00e+00  2.70e+05        1    1.94e-03    7.46e-03
Ceres Solver Report: Iterations: 4, Initial cost: 1.074601e+05, Final cost: 2.083842e-14, Termination: CONVERGENCE


## Optimize poses

In [21]:
def define_problem2(rec):
    prob = pyceres.Problem()
    loss = pyceres.TrivialLoss()
    for im in rec.images.values():
        cam = rec.cameras[im.camera_id]
        for p in im.points2D:
            cost = pycolmap.cost_functions.ReprojErrorCost(cam.model, p.xy)
            pose = im.cam_from_world
            params = [
                pose.rotation.quat,
                pose.translation,
                rec.points3D[p.point3D_id].xyz,
                cam.params,
            ]
            prob.add_residual_block(cost, loss, params)
        prob.set_manifold(
            im.cam_from_world.rotation.quat, pyceres.EigenQuaternionManifold()
        )
    for cam in rec.cameras.values():
        prob.set_parameter_block_constant(cam.params)
    for p in rec.points3D.values():
        prob.set_parameter_block_constant(p.xyz)
    return prob


rec = create_reconstruction()
for im in rec.images.values():
    im.cam_from_world.translation += np.random.randn(3) / 2
    im.cam_from_world.rotation *= pycolmap.Rotation3d(np.random.randn(3) / 5)
    im.cam_from_world.rotation.normalize()
rec_init = rec.__deepcopy__({})
init_from_gt = [
    rec.images[i].cam_from_world * rec_gt.images[i].cam_from_world.inverse()
    for i in rec.images
]
print([np.linalg.norm(t.translation) for t in init_from_gt])
print([np.rad2deg(t.rotation.angle()) for t in init_from_gt])
problem = define_problem2(rec)
solve(problem)
opt_from_gt = [
    rec.images[i].cam_from_world * rec_gt.images[i].cam_from_world.inverse()
    for i in rec.images
]
print([np.linalg.norm(t.translation) for t in opt_from_gt])
print([np.rad2deg(t.rotation.angle()) for t in opt_from_gt])

[np.float64(1.1219738478901715), np.float64(0.29153152220166934)]
[np.float64(32.546967105288715), np.float64(16.036538808692125)]
55 167 100 200
iter      cost      cost_change  |gradient|   |step|    tr_ratio  tr_radius  ls_iter  iter_time  total_time
   0  5.661147e+05    0.00e+00    1.00e+06   0.00e+00   0.00e+00  1.00e+04        0    2.16e-04    6.86e-04
   1  1.589775e+04    5.50e+05    1.22e+05   8.21e-01   9.73e-01  3.00e+04        1    4.77e-04    1.19e-03
   2  2.077984e+02    1.57e+04    2.08e+04   4.43e-01   9.87e-01  9.00e+04        1    2.81e-04    1.49e-03
   3  3.067690e-02    2.08e+02    2.50e+02   4.13e-02   1.00e+00  2.70e+05        1    2.62e-04    1.77e-03
   4  8.020149e-10    3.07e-02    2.25e-01   5.20e-04   1.00e+00  8.10e+05        1    2.70e-04    2.06e-03
   5  1.703731e-19    8.02e-10    2.99e-07   9.13e-08   1.00e+00  2.43e+06        1    2.79e-04    2.34e-03
Ceres Solver Report: Iterations: 6, Initial cost: 5.661147e+05, Final cost: 1.703731e-19, Terminat

In [22]:
assert np.allclose(rec.cameras[0].params, rec_gt.cameras[0].params)
for i in rec.images:
    print(
        rec.images[i].cam_from_world.translation,
        rec_gt.images[i].cam_from_world.translation,
    )
    print(
        rec.images[i].cam_from_world.rotation.quat,
        rec_gt.images[i].cam_from_world.rotation.quat,
    )
rec.points3D[1].xyz, rec_gt.points3D[1].xyz

[0.80142173 0.33757516 0.57766469] [0.80142173 0.33757516 0.57766469]
[-7.14886602e-14  2.90551251e-13  2.14872540e-14  1.00000000e+00] [0. 0. 0. 1.]
[-0.37236177 -0.16083449  0.75442408] [-0.37236177 -0.16083449  0.75442408]
[-7.23833806e-16  1.88096324e-16 -6.69714114e-17  1.00000000e+00] [0. 0. 0. 1.]


(array([0.10159581, 0.41629565, 2.58180948]),
 array([0.10159581, 0.41629565, 2.58180948]))

In [23]:
fig = viz_3d.init_figure()
viz_3d.plot_reconstruction(
    fig,
    rec_init,
    min_track_length=0,
    color="rgb(255,255,255)",
    points_rgb=False,
    name="init",
)
viz_3d.plot_reconstruction(
    fig, rec_gt, min_track_length=0, color="rgb(255,0,0)", points_rgb=False, name="GT"
)
viz_3d.plot_reconstruction(
    fig,
    rec,
    min_track_length=0,
    color="rgb(0,255,0)",
    points_rgb=False,
    name="optimized",
)
fig.show()